# Enterprise AI Search (ICT CoE) 

Solution that empowers customer-facing teams to access the knowledge they need to respond to sales inquiries, support questions, and other client communication while on the go.

-	Device Catalogue

-	Product Catalogue - Cyber Security, Cloud, Applications - IoT

-	Finance Reporting

-	Customer Support - Smart Water

-	Procurement

-	Tender

This Next-generation enterprise search will help you instantly find the answers you need.

## Install

In [2]:
!pip install -qU  llama-index==0.10.64

In [3]:
!pip install -qU llama-index-core==0.10.64

In [4]:
!pip install -qU  llama-index-llms-bedrock==0.1.12

In [5]:
!pip install -qU llama-index-embeddings-huggingface==0.2.3

## Setup

In [6]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [7]:
import nest_asyncio

nest_asyncio.apply()

In [8]:
import os
import logging
import boto3

from llama_index.core import SimpleDirectoryReader, ServiceContext

from llama_index.core import SimpleKeywordTableIndex
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core import VectorStoreIndex,SummaryIndex

from llama_index.llms.bedrock import Bedrock

## LLM and embedding model configuration

In [9]:
instruct_mistral7b_id="mistral.mistral-7b-instruct-v0:2"

instruct_mixtral8x7b_id="mistral.mixtral-8x7b-instruct-v0:1"
mistral_large_2402_id="mistral.mistral-large-2402-v1:0"
titan_embeddings_g1="amazon.titan-embed-text-v1"
titan_text_embeddings_v2="amazon.titan-embed-text-v2:0"


DEFAULT_MODEL=instruct_mistral7b_id
AWS_REGION="eu-west-1"

boto3_bedrock = boto3.client("bedrock-runtime")

model_kwargs_mistral = {
    "temperature": 0.5,
    "top_p": 0.9,
    "top_k": 200,
    "max_tokens": 8192  # Max response length
}

# Initialize the Mistral model to formulate final answer from search results
llm = Bedrock(
    model=DEFAULT_MODEL,
    streaming=True,
    client=boto3_bedrock,
    model_kwargs=model_kwargs_mistral,
    region_name=AWS_REGION
)

In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = llm
Settings.chunk_size = 512
Settings.embed_model = embed_model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Data loading and processing

In [11]:
import os

# Path to your PDF directory
pdf_directory = 'coe_data'

# List all PDF files in the directory
pdf_files = [file for file in os.listdir(pdf_directory) if file.endswith('.pdf')]

# Create a Dictionary to hold the loaded documents
loaded_documents = {}

# Initialize a list to hold the titles of the PDFs
pdf_titles = []

# Function to convert a title to snake case
def convert_to_snake_case(title):
    return title.lower().replace(' ', '_')

# Load each PDF file, convert the title to snake case, and store it in the dictionary
for pdf_file in pdf_files:
    
    # Extract the title without the .pdf extension and convert to snake case
    title = os.path.splitext(pdf_file)[0]
    snake_case_title = convert_to_snake_case(title)
    pdf_titles.append(snake_case_title)
    
    # Load the document using SimpleDirectoryReader
    document_path = os.path.join(pdf_directory, pdf_file)
    documents = SimpleDirectoryReader(input_files=[document_path]).load_data()
    
    # Store the loaded documents in the dictionary with the snake_case title as the key
    loaded_documents[snake_case_title] = documents

# Loaded_documents dictionary holds the content of each PDF file keyed by its title converted to snake case

In [12]:
pdf_titles

['how_to_sell_aws_services',
 'endpoint_security_proposition_-_refresher_slides',
 'telematics_offering_-_revised',
 'how_to_sell_safaricom_cloud_&_hosting']

In [13]:
from llama_index.core.agent import ReActAgent
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter

node_parser = SentenceSplitter()

# Build agents dictionary
agents = {}

# Iterate over the snake_case titles in loaded_documents
for snake_case_title in loaded_documents.keys():
    # Retrieve the documents from the dictionary
    documents = loaded_documents[snake_case_title]
    
    # nodes = node_parser.get_nodes_from_documents(documents)
      
    if not os.path.exists(f"./coe_multi/{snake_case_title}"):
        # build vector index
        vector_index = VectorStoreIndex(documents)
        vector_index.storage_context.persist(
        persist_dir=f"./coe_multi/{snake_case_title}"
        )
    else:
        vector_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir=f"./coe_multi/{snake_case_title}"),
        )       
        
    # Build vector index
    # vector_index = VectorStoreIndex.from_documents(documents)
    
    # Build summary index
    summary_index = SummaryIndex.from_documents(documents)
    
    # Define query engines
    vector_query_engine = vector_index.as_query_engine()
    summary_query_engine = summary_index.as_query_engine()

    # Define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    f"Useful for retrieving specific context from {snake_case_title}"
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    "Useful for summarization questions related to"
                    f" {snake_case_title}"
                ),
            ),
        ),
    ]

    # Build agent
    agent = ReActAgent.from_tools(
        query_engine_tools,
        llm=llm,  
        verbose=True,
            system_prompt=f"""\
You are a specialized agent designed to answer queries about the {snake_case_title}.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
    )

    # Store the agent in the dictionary with the snake_case title as the key
    agents[snake_case_title] = agent


## Define IndexNode for each of these Agents

In [14]:
from llama_index.core.schema import IndexNode

# Define top-level nodes
objects = []

for snake_case_title in loaded_documents.keys():
    # Define a summary for each document
    document_summary = (
        f"This content contains information related to {snake_case_title.replace('_', ' ')}. "
        "Use this index if you need to lookup specific facts about "
        f"{snake_case_title.replace('_', ' ')}.\nDo not use this index if you want to analyze "
        "multiple topics."
    )
    
    # Define the index node that links to these agents
    node = IndexNode(
        text=document_summary, index_id=snake_case_title, obj=agents[snake_case_title]
    )
    objects.append(node)


In [15]:
vector_index = VectorStoreIndex(
    objects=objects,
)
query_engine = vector_index.as_query_engine(similarity_top_k=1, verbose=True)

## Test Queries
Should choose a vector tool/ summary tool for a specific agent based on the query.

In [16]:
# should use Telematics agent -> vector tool
response = query_engine.query("What is the Safaricom Telematics?")

Retrieval entering telematics_offering_-_revised: ReActAgent
Retrieving from object ReActAgent with query What is the Safaricom Telematics?
> Running step f19bdc04-fa64-42f2-b00f-fc0637c3db7a. Step input: What is the Safaricom Telematics?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool
Action Input: {'input': 'Safaricom Telematics'}
Observation:  Safaricom Telematics is a service offering that provides vehicle management solutions using telematics technology. The details of this offering, including its coverage, can be found in the document "Telematics Offering - Revised.pdf" on page 1 and page 6.
> Running step 6901934e-3e86-43b3-906f-c4ac8398443b. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Safaricom Telematics is a service offering that provides vehicle management solutions using telematics technology. For more details, you can refer

In [17]:
# should use End Point Srcurity agent -> vector tool
response = query_engine.query("Why Endpoint Security?")

Retrieval entering endpoint_security_proposition_-_refresher_slides: ReActAgent
Retrieving from object ReActAgent with query Why Endpoint Security?
> Running step 8904e0b4-50f3-4104-98e3-02b7ea287e13. Step input: Why Endpoint Security?
Thought: The current language of the user is: english. I need to use a tool to help me answer the question "Why Endpoint Security?".
Action: summary_tool
Action Input: {'input': 'Why Endpoint Security?'}
Observation:  Endpoint security is essential for protecting devices like laptops, desktops, tablets, and smartphones from cyberattacks such as malware, ransomware, and unauthorized access. With the increasing use of mobile devices in enterprises, organizations are seeking effective solutions to secure and manage these devices to ensure data privacy and regulatory compliance. The need for endpoint security is further emphasized by the rise in remote work and bring-your-own-device (BYOD) policies, which expand the attack surface and increase the risk of da

In [18]:
# should use Safaricom Cloud agent -> vector tool
response = query_engine.query("What have we got to say about Safaricom Cloud & Hosting Services?") 

Retrieval entering how_to_sell_safaricom_cloud_&_hosting: ReActAgent
Retrieving from object ReActAgent with query What have we got to say about Safaricom Cloud & Hosting Services?
> Running step a26a2fcb-2e52-474f-bc50-ce76329946c4. Step input: What have we got to say about Safaricom Cloud & Hosting Services?
Thought: The current language of the user is: English. I need to use a tool to help me retrieve specific context about Safaricom Cloud & Hosting Services.
Action: vector_tool
Action Input: {'input': 'Safaricom Cloud & Hosting Services'}
Observation:  Safaricom Cloud & Hosting Services offer enterprise-class solutions that enable organizations to manage IT infrastructure growth and take advantage of a more flexible and cost-efficient IT infrastructure. They provide a range of services including co-location, private cloud, flexible computing, storage, and security. Co-location offers secure data center resources for a predictable monthly rental, while private cloud provides a dedica

In [29]:
# should use How to sell AWS End Services agent -> vector tool
response = query_engine.query("Why Endpoint Security – COBO & COPE") 

Retrieval entering endpoint_security_proposition_-_refresher_slides: ReActAgent
Retrieving from object ReActAgent with query Why Endpoint Security – COBO & COPE
> Running step b23007a5-987a-45a3-874e-1189a21aed3f. Step input: Why Endpoint Security – COBO & COPE
Thought: The current language of the user is: english. I need to use a vector_tool to help me understand the context of the question before I can determine which tool to use next.
Action: vector_tool
Action Input: {'input': 'Why Endpoint Security – COBO & COPE'}
Observation:  Endpoint security is essential in today's business environment due to the increasing use of mobile devices in enterprises, particularly with the Bring Your Own Device (BYOD) trend and remote work arrangements. These factors expand the attack surface and make it crucial to secure and manage devices to maintain data privacy and regulatory compliance. Endpoint security solutions help protect against data breaches caused by malware and phishing attempts, which 

In [19]:
# should use How to sell AWS End Services agent -> vector tool
response = query_engine.query("How can public cloud help businesses?") 

Retrieval entering how_to_sell_aws_services: ReActAgent
Retrieving from object ReActAgent with query How can public cloud help businesses?
> Running step 7efb7eb7-a559-4a14-892a-a332407ee1e6. Step input: How can public cloud help businesses?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: summary_tool
Action Input: {'input': 'How can public cloud help businesses?'}
Observation:  Public cloud can help businesses in several ways. It offers infrastructure that can be easily scaled up and down as demand changes, providing specific performance and availability required by applications. Public clouds enable businesses to deliver results faster, cheaper, and with higher quality, giving them a competitive edge and enabling them to adapt and get to market quickly. They provide consistent services globally, enabling businesses to easily deploy pre-built, consistent services, server images, application services, or entire applica

In [20]:
response.response

' Public cloud provides businesses with scalable infrastructure that can be easily adjusted to meet changing demands. It offers consistent services globally, enabling faster and cheaper delivery of results with higher quality. The pay-as-you-go cost model allows businesses to only pay for what they use, while managed services and cloud management tools simplify IT operations.'

In [21]:
## Persisiting on Disk
## Convert fles in py for deployment
## Improve Streamlit app - include audio and clear
## Add a reranker
## Add sentece splitters for efficient indexing, improved context understanding, better query matching, flexibility in retrieval strategies, and scalability. 

In [30]:
# should use How to sell AWS End Services agent -> vector tool
response = query_engine.query("CEBO in Cloud?")

Retrieval entering how_to_sell_safaricom_cloud_&_hosting: ReActAgent
Retrieving from object ReActAgent with query CEBO in Cloud?
> Running step b637626e-f14d-427a-87ad-bb0cc4223d1a. Step input: CEBO in Cloud?
Thought: The user is asking about CEBO in the context of Safaricom Cloud. I need to use the vector_tool to retrieve specific context about CEBO in Safaricom Cloud.
Action: vector_tool
Action Input: {'input': 'CEBO in Safaricom Cloud'}
Observation:  CEBO, which stands for Cloud and Business Operations, is a service offering by Safaricom that provides enterprise-class cloud and hosting solutions to organizations. The service enables businesses to manage their growth in computing and data while taking advantage of a more flexible and cost-efficient IT infrastructure. Safaricom offers various solutions under CEBO, including Co-location, Private Cloud, Flexible Computing, Storage, and Security. These solutions aim to help businesses reduce IT infrastructure costs, improve service agili

In [32]:
# should use How to sell AWS End Services agent -> vector tool
response = query_engine.query("CEBO in Telematics?")

Retrieval entering telematics_offering_-_revised: ReActAgent
Retrieving from object ReActAgent with query CEBO in Telematics?
> Running step 269080a6-1f04-4802-9a4f-34bfbe79fb94. Step input: CEBO in Telematics?
Thought: The current language of the user is: english. I need to use a tool to help me understand the context of CEBO in the context of telematics.
Action: vector_tool
Action Input: {'input': 'CEBO in Telematics?'}
Observation:  CEBO, which stands for Centralized Business Operations, is a concept that refers to managing and coordinating various business functions and processes from a single, centralized location. In the context of Telematics, CEBO could potentially be used to manage and analyze data from multiple vehicles or fleets in real-time, providing insights and optimizing operations. However, without additional context from the provided document, it is impossible to determine if CEBO is specifically mentioned or implemented in the Telematics offering discussed in the docu

In [2]:
## TODO: Add get_embedding as a Lamda function that gets 
#triggered when new docs are added to sharepoint / S3 bucket. New embeddings created
#Add sentence splitter
## Add Memory

# https://docs.llamaindex.ai/en/stable/api_reference/node_parsers/sentence_splitter/